## Selenium 사용 동적 크롤링 - 네이버 뉴스

In [1]:
# Selenium 설치.
!pip install selenium

In [2]:
from selenium import webdriver
import bs4
import re
import os  #이부분 새로 작성    #cromedriver.exe새로 깔았음

In [3]:
os.chdir(r"C:\Users\Gram\Desktop\myPyCode\02  데이터 수집과 처리 - 실습") 

### 1. 사이트에 들어가서 탐색해 본다:

1. 먼저 다음 사이트에 들어가서 살펴본다:  [네이버 뉴스](https://news.naver.com)
2. 크롬에서 3점 메뉴 버튼을 누르고 **도구 더보기** ==> **개발자 도구**를 선택해서 개발자 도구를 연다.
3. 개발자 도구 왼쪽 상단의 화살표 버튼을 눌러서 사이트 일부를 서택하고 해당 HTML코드를 볼 수 있다. "[CTRL] + [SHIFT] + C" 조합으로 대체할 수 있다.

### 2. 이제는 Selenium을 사용해서 내용을 가져와 본다:

In [4]:
# 웹드라이버 로딩.
my_driver = webdriver.Chrome('chromedriver.exe') 

In [5]:
# 사이트에 접속.
my_driver.get("https://news.naver.com")

In [6]:
# CSS 셀렉터를 사용하여 헤드라인 타이틀 가져오기. 
res = my_driver.find_elements_by_css_selector('div.hdline_article_tit')
for x in res:
    print(x.text.strip())

안철수 "보수통합 관심없다, 출마 안해"…제3의 길 공식화
'실종 사흘째' 안나푸르나 헬기 띄워 수색…"관련 흔적 못찾아"(종합)
세입자 궁지로 모는 규제…전셋값 자극할듯
돌아온 安 "중도실용정당 창당…보수통합 관심없다"
'깃발만 꽂으면 당선' 한국당 TK 첫 불출마 선언한 정종섭


### 3. Parsing 부분은 BeautifulSoup로 처리하면 간편하다: Selenium은 iframe경우만 쓰자

In [7]:
my_html = my_driver.page_source                      # 사이트 내용을 가져와서 BeautifulSoup로 처리
soup = bs4.BeautifulSoup(my_html, 'html.parser')     # BeautifulSoup 객체 반환. 'html.parser' 사용.

링크 타이틀 중에서 볼드 처리 뉴스 타이틀과 하이퍼링크 가져오기:

In [8]:
res_a = soup.find_all('a', class_=re.compile('clicks'))      # "clicks" 패턴 있는 클래스.
n_count = 0
news = []
for x in res_a:
    res = x.find('strong')                                   # 다시 검색.
    if res:                                                  # None이 아닌 경우만.
        an_article = res.text
        if (len(an_article) > 10):                           # 길이가 10 이상인 경우만.
            n_count += 1
            print(n_count, end = " : ")
            print(an_article, end = " => ")
            print(x.get('href'))
            news += [(an_article, x.get("href"))]

1 : [카톡방담] 대통령, 사회 실력 뽐냈지만 북한... => /main/list.nhn?mode=LPOD&mid=sec&oid=469&listType=paper&date=20200119
2 : 이동섭 "손학규, 물러나지 않으면 신당 창당 할 수 밖에 없어" => https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=421&aid=0004416273
3 : 정부, 호르무즈 해협 ‘독자적 파병’으로 가닥 => https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=028&aid=0002482170
4 : ‘미국통’ 리용호 경질, 강경파 리선권 외무상에…김정은 속내는? => https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=020&aid=0003265102
5 : [TF현장] "사랑해요 안철수!"…安 "더욱 간절해졌다" => https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=629&aid=0000011152
6 : "새정치" 공항 가득 채운 각양각색 기대 …'큰절' 화답한 안철수 => https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=421&aid=0004416269
7 : ‘혈혈단신’ 대한해협 건너 롯데그룹 일군 신격호 명예회장 별세 => https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=101&oid=021&aid=0002414484
8 : 네번째 국토부 2차관, 이번엔 다를까 => https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=101&oid=014&aid=0004359810
9 : [단독] 도로·철도 사업비 2